In [18]:
print("hello")


hello


## project started 
Demo time

In [19]:
import os
import requests
from dotenv import load_dotenv
import json
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr
from datetime import datetime
import sys

In [20]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [21]:
model="gpt-4o-mini"
openai = OpenAI()
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')

In [22]:
system_prompt = """You are FarmSakha AI – a multilingual, AI-powered Smart Crop Advisory Assistant designed to help small and marginal farmers in India. 
Your role is to provide personalized, real-time agricultural guidance in simple and easy-to-understand language.

Core Responsibilities:
1. Crop Advisory:
   - Suggest suitable crops based on soil type, weather conditions, season, and historical crop data.
   - Provide scientific fertilizer and irrigation recommendations with correct NPK ratios.

2. Pest and Disease Management:
   - Help farmers identify pests/diseases through image descriptions or text inputs.
   - Suggest eco-friendly and cost-effective solutions, including biological and chemical control methods.

3. Weather & Risk Alerts:
   - Provide localized, real-time weather updates and 7-day forecasts using coordinates.
   - Warn about risks like drought, floods, or pest outbreaks and suggest preventive measures.
   - Help farmers plan irrigation, spraying, and harvesting based on weather conditions.

4. Soil Health:
   - Give soil test interpretation and fertilizer/lime/organic manure guidance.
   - Promote sustainable practices that reduce chemical overuse.

5. Market & Economics:
   - Share local mandi/market prices of crops using real-time search.
   - Provide insights on which crops may yield better income for the season.

6. Accessibility:
   - Support answers in multiple Indian languages.
   - Provide step-by-step, voice-friendly, and simple responses for low-literate farmers.

Tools Available:
- Real-time Google search for market prices, news, and current information
- Weather forecasting using city name (Weather_forecast)

When farmers ask about weather:
- If they provide place name, use the weather tool (Weather_forecast) directly with city name as input or ask them the city name.
- Always relate weather information to farming activities (when to irrigate, spray, harvest, etc.)
- Dont give whole hardcoded detail of weather forecast only important things and sum ups, suggestions, conclusion etc. like a smart friend.
- give details if required.

Important Guidelines:
- Always prioritize sustainable, cost-effective, and farmer-friendly practices.
- Avoid overly technical jargon – explain in simple words with examples.
- When uncertain, advise consulting local agriculture officers or Krishi Vigyan Kendras.
- Ensure trustworthiness, cultural sensitivity, and clarity in communication.

Your mission: Empower small and marginal farmers with actionable insights, 
improve productivity, reduce costs, and promote environmentally friendly farming practices.


"""


In [23]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

In [24]:
def get_Google_Search(query: str):
        url = "https://api.tavily.com/search"
        headers = {"Content-Type": "application/json"}
        payload = {
            "api_key":TAVILY_API_KEY ,
            "query": query,
            "search_depth": "basic",
            "include_answer": True
        }
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        answer = data.get("answer", "")
        results = "\n".join(
            [f"- {r['title']} ({r['url']})" for r in data.get("results", [])]
        )
        return f"{answer}\n\nSources:\n{results}"

get_Google_Search("what is the price of wheat today in india")

'As of today, the price of wheat in India is ₹2480 per quintal, with variations in different regions. In Uttar Pradesh, 1 kg of wheat costs ₹25.25. These prices are current as of 2025-09-09.\n\nSources:\n- Wheat Price Today in India - 09 Sep 2025 - BankBazaar (https://www.bankbazaar.com/commodity-price/wheat-price.html)\n- Global Wheat Price | Tridge (https://dir.tridge.com/prices/wheat)\n- Wheat mandi price - Commodity Online (https://www.commodityonline.com/mandiprices/wheat)\n- Wheat Futures Price Today - Investing.com India (https://in.investing.com/commodities/ncdex-wheat-futures)\n- Wheat mandi price today in Uttar Pradesh - Commodity Online (https://www.commodityonline.com/mandiprices/wheat/uttar-pradesh)'

In [25]:
# There's a particular dictionary structure that's required to describe our function:

Google_Search = {
    "name": "get_Google_Search",
    "description": "Search Google for real-time information (news, crops, weather, market prices, etc.)",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query string",
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

weather forcast


In [26]:
def get_Weather_forecast(city: str):
    """
    Fetch detailed weather forecast for a given latitude and longitude
    with comprehensive farming-related parameters.
    
    Returns structured data (dict) instead of fixed advice text, 
    so that an LLM can analyze and generate context-specific recommendations.
    """


    # Step 1: Geocode city to get latitude/longitude
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    geo_resp = requests.get(geo_url)
    geo_data = geo_resp.json()

    if "results" not in geo_data:
        print(f"City '{city}' not found.")
        exit()

    latitude = geo_data["results"][0]["latitude"]
    longitude = geo_data["results"][0]["longitude"]
    # print(f"Coordinates for {city}: {lat}, {lon}")

    try:
        weather_url = "https://api.open-meteo.com/v1/forecast"
        weather_params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": [
                "temperature_2m_max", "temperature_2m_min",
                "apparent_temperature_max", "apparent_temperature_min",
                "precipitation_sum", "rain_sum", "snowfall_sum",
                "windspeed_10m_max", "windgusts_10m_max",
                "winddirection_10m_dominant",
                "sunrise", "sunset", "daylight_duration",
                "et0_fao_evapotranspiration", "precipitation_probability_max",
                "shortwave_radiation_sum", "uv_index_max"
            ],
            "hourly": [
                "temperature_2m", "relative_humidity_2m", "dewpoint_2m",
                "apparent_temperature", "precipitation", "rain", "snowfall",
                "cloudcover", "cloudcover_low", "cloudcover_mid", "cloudcover_high",
                "windspeed_10m", "windgusts_10m", "winddirection_10m",
                "surface_pressure", "visibility", "soil_moisture_0_1cm",
                "soil_moisture_1_3cm", "soil_moisture_3_9cm", "soil_moisture_9_27cm",
                "soil_temperature_0cm", "soil_temperature_6cm", "soil_temperature_18cm",
                "et0_fao_evapotranspiration", "shortwave_radiation", "uv_index",
                "precipitation_probability"
            ],
            "forecast_days": 7,
            "timezone": "auto"
        }
        
        response = requests.get(weather_url, params=weather_params)
        weather_data = response.json()

        if "daily" not in weather_data:
            return {"error": "Weather data not available for the given coordinates."}

        # Prepare structured output
        output = {
            "location": {"latitude": latitude, "longitude": longitude},
            "daily_forecast": [],
            "hourly_forecast_today": [],
            "weekly_summary": {}
        }

        # Helper: wind direction conversion
        def get_wind_direction(degrees):
            if degrees is None:
                return "Variable"
            directions = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE",
                          "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
            return directions[int((degrees + 11.25) % 360 / 22.5)]

        # Daily forecast (7 days)
        daily = weather_data["daily"]
        for i in range(min(7, len(daily["time"]))):
            date = daily["time"][i]
            date_obj = datetime.fromisoformat(date)
            day_name = date_obj.strftime('%A')

            output["daily_forecast"].append({
                "date": date,
                "day": day_name,
                "temperature": {
                    "min": daily["temperature_2m_min"][i],
                    "max": daily["temperature_2m_max"][i],
                    "apparent_min": daily["apparent_temperature_min"][i],
                    "apparent_max": daily["apparent_temperature_max"][i],
                },
                "precipitation": {
                    "rain": daily["rain_sum"][i] or 0,
                    "total": daily["precipitation_sum"][i] or 0,
                    "probability": daily["precipitation_probability_max"][i] or 0,
                },
                "wind": {
                    "speed_max": daily["windspeed_10m_max"][i],
                    "gusts_max": daily["windgusts_10m_max"][i],
                    "direction_deg": daily["winddirection_10m_dominant"][i],
                    "direction_cardinal": get_wind_direction(daily["winddirection_10m_dominant"][i]),
                },
                "sun": {
                    "sunrise": daily["sunrise"][i],
                    "sunset": daily["sunset"][i],
                    "daylight_hours": round(daily["daylight_duration"][i] / 3600, 1) if daily["daylight_duration"][i] else None,
                },
                "evapotranspiration": daily["et0_fao_evapotranspiration"][i] or 0,
                "radiation": daily["shortwave_radiation_sum"][i] or 0,
                "uv_index_max": daily["uv_index_max"][i] or 0,
            })

        # Hourly forecast for today (24 hours)
        hourly = weather_data["hourly"]
        for i in range(min(24, len(hourly["time"]))):
            dt = datetime.fromisoformat(hourly["time"][i].replace('Z', '+00:00'))
            time_str = dt.strftime('%H:%M')

            output["hourly_forecast_today"].append({
                "time": time_str,
                "temperature": hourly["temperature_2m"][i],
                "apparent_temperature": hourly["apparent_temperature"][i],
                "humidity": hourly["relative_humidity_2m"][i],
                "dewpoint": hourly["dewpoint_2m"][i],
                "precipitation": {
                    "rain": hourly["rain"][i] or 0,
                    "probability": hourly["precipitation_probability"][i] or 0,
                },
                "cloudcover": hourly["cloudcover"][i],
                "wind": {
                    "speed": hourly["windspeed_10m"][i],
                    "gusts": hourly["windgusts_10m"][i],
                    "direction_deg": hourly["winddirection_10m"][i],
                },
                "pressure": hourly["surface_pressure"][i],
                "visibility_km": (hourly["visibility"][i] / 1000) if hourly["visibility"][i] else None,
                "soil": {
                    "temperature_surface": hourly["soil_temperature_0cm"][i],
                    "temperature_6cm": hourly["soil_temperature_6cm"][i],
                    "moisture_surface": hourly["soil_moisture_0_1cm"][i] or 0,
                    "moisture_3_9cm": hourly["soil_moisture_3_9cm"][i] or 0,
                },
                "evapotranspiration": hourly["et0_fao_evapotranspiration"][i] or 0,
                "uv_index": hourly["uv_index"][i] or 0,
            })

        # Weekly summary
        total_rain = sum([daily["rain_sum"][i] or 0 for i in range(min(7, len(daily["time"])))])
        avg_temp_max = sum([daily["temperature_2m_max"][i] for i in range(min(7, len(daily["time"])))]) / min(7, len(daily["time"]))
        avg_temp_min = sum([daily["temperature_2m_min"][i] for i in range(min(7, len(daily["time"])))]) / min(7, len(daily["time"]))
        total_evaporation = sum([daily["et0_fao_evapotranspiration"][i] or 0 for i in range(min(7, len(daily["time"])))])
        water_balance = total_rain - total_evaporation

        output["weekly_summary"] = {
            "total_rain": total_rain,
            "avg_temp_min": avg_temp_min,
            "avg_temp_max": avg_temp_max,
            "total_evapotranspiration": total_evaporation,
            "water_balance": water_balance
        }

        return output

    except Exception as e:
        return {"error": str(e)}

    
# print(get_Weather_forecast("punjab"))

In [27]:
# There's a particular dictionary structure that's required to describe our function:

Weather_forecast = {
    "name": "get_Weather_forecast",
    "description": "Fetch weather forecast (hourly for today and daily for next 14 days) for a given City (farmer location)",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {"type": "string", "description": "Name of the city"}
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [28]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": Google_Search},
        {"type":"function","function":Weather_forecast}]

In [29]:
def handle_tool_call(message):
    """Handle tool calls from OpenAI"""
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    if function_name == "get_Google_Search":
        query = arguments.get('query')
        search_result = get_Google_Search(query)
        result_content = search_result
        
    elif function_name == "get_Weather_forecast":
        city = arguments.get('city')
        weather_result = get_Weather_forecast(city)
        result_content = json.dumps(weather_result)
        
    else:
        result_content = f"Unknown function: {function_name}"
        print(result_content)
    
    response = {
        "role": "tool",
        "content": result_content,
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
def chat(message, history):
    """Main chat function for Gradio interface"""
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_prompt}]
    
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "content": human_msg})
        if ai_msg:  # Only add if AI response exists
            messages.append({"role": "assistant", "content": ai_msg})
    
    messages.append({"role": "user", "content": message})
    
    # Make initial API call
    response = openai.chat.completions.create(
        model=model, 
        messages=messages, 
        tools=tools,
        # stream=True
    )

    # Handle tool calls if present
    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        print("tool called:",assistant_message)
        tool_response = handle_tool_call(assistant_message)
        
        # Add tool call and response to messages
        messages.append({
            "role": "assistant",
            "tool_calls": assistant_message.tool_calls
        })
        messages.append(tool_response)
        
        # Make final API call
        final_response = openai.chat.completions.create(
            model=model, 
            messages=messages
        )
        return final_response.choices[0].message.content
    
    return response.choices[0].message.content

In [ ]:
# Launch Gradio interface
if __name__ == "__main__":
    interface = gr.ChatInterface(
        fn=chat,
        title="🌾 FarmSakha AI - Smart Crop Advisory Assistant",
        description="Ask me anything about farming, crops, weather, market prices, or agricultural practices in India!",
        examples=[
            "What crops are best for clay soil in Punjab?",
            "Current wheat prices in India",
            "How to control aphids in cotton crop?",
            "Weather forecast for farming this week"
        ]
    )
    interface.launch()


c:\SIH\FarmSakha-AI\venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\SIH\FarmSakha-AI\venv\Lib\site-packages\gradio\chat_interface.py", line 959, in _stream_fn
    first_response = await utils.async_iteration(generator)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\SIH\FarmSakha-AI\venv\Lib\site-packages\gradio\utils.py", line 760, in async_iteration
    return await anext(iterator)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\SIH\FarmSakha-AI\venv\Lib\site-packages\gradio\utils.py", line 751, in __anext__
    return await anyio.to_thread.run_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        run_sync_iterator_async, self.iterator, limiter=self.limiter
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\SIH\FarmSakha-AI\venv\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        func, args, abandon_on